In [65]:
import numpy as np
import pandas as pd

In [71]:
aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')
order_products__prior = pd.read_csv('data/order_products__prior.csv')
order_products__train = pd.read_csv('data/order_products__train.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [67]:
# given prior, predict things in test (scoring https://www.kaggle.com/c/instacart-market-basket-analysis#evaluation)
# cluster different buy patterns for items and customers

In [78]:
np.unique(orders.user_id)

array([     1,      2,      3, ..., 206207, 206208, 206209])

In [79]:
np.max(orders.user_id)

206209

In [80]:
len(np.unique(orders.user_id))

206209

In [73]:
def predict(orders_subset):
    orders_subset = orders_subset.sort('order_id')
    
    predicted_orders = [(-1) for i in range(len(orders_subset))]
    return predicted_orders

def predict_and_test(orders_subset, order_products):
    def aggregate_rows(rows):
        if sum(rows.reordered) == 0:
            return (-1)
        filter_products = [product for reordered, product in zip(rows.reordered, rows.product_id) if reordered]
        return (filter_products)

    def score_row(real_order, predicted_order):
        correct_elements = float(len([product for product in predicted_order if product in real_order]))
        precision = correct_elements / len(predicted_order)
        recall = correct_elements / len(real_order)
        return 2 * (precision * recall) / (precision + recall) if correct_elements else 0
    
    # Get rel answers
    grouped = order_products.groupby(order_products.order_id)
    products_by_order = grouped.apply(func=aggregate_rows)
    real_orders = products_by_order.sort_index().values
    
    # Get predicted answers
    predicted_orders = predict(orders_subset)
    
    # Get score
    score = sum(score_row(real, predicted) for real, predicted in zip(real_orders, predicted_orders)) / len(real_orders)
    print('Final Score: ', score)

In [75]:
orders_prior = orders[orders.eval_set == 'prior']
predict_and_test(orders_prior, order_products__prior)

orders_train = orders[orders.eval_set == 'train']
predict_and_test(orders_train, order_products__train)

KeyboardInterrupt: 